In [1]:
from dapr_agents import OpenAPIReActAgent
from dapr_agents.tool.utils import OpenAPISpecParser
from dapr_agents.document.embedder import SentenceTransformerEmbedder
from dapr_agents.storage import ChromaVectorStore
from dotenv import load_dotenv
import logging

In [2]:
logging.basicConfig(level=logging.INFO)

load_dotenv()

True

In [3]:
spec_parser = OpenAPISpecParser.from_url("http://127.0.0.1:8000/openapi.json")

In [4]:
spec_parser.endpoints[0]

('GET /health',
 'Return the health status of the API.',
 Operation(tags=['system'], summary='Health check endpoint', description='Return the health status of the API.', externalDocs=None, operationId='health_check_health_get', parameters=None, requestBody=None, responses={'200': Response(description='Successful Response', headers=None, content={'application/json': MediaType(media_type_schema=Schema(allOf=None, anyOf=None, oneOf=None, schema_not=None, schema_if=None, then=None, schema_else=None, dependentSchemas=None, prefixItems=None, items=None, contains=None, properties=None, patternProperties=None, additionalProperties=Schema(allOf=None, anyOf=None, oneOf=None, schema_not=None, schema_if=None, then=None, schema_else=None, dependentSchemas=None, prefixItems=None, items=None, contains=None, properties=None, patternProperties=None, additionalProperties=None, propertyNames=None, unevaluatedItems=None, unevaluatedProperties=None, type=<DataType.STRING: 'string'>, enum=None, const=None, 

In [5]:
len(spec_parser.endpoints)

5

In [6]:
from dapr_agents.tool.utils.openapi import openapi_spec_to_openai_fn

functions = openapi_spec_to_openai_fn(spec_parser)

In [7]:
functions[1]

{'definition': {'type': 'function',
  'function': {'name': 'add',
   'description': 'Add two numbers and return the result.',
   'parameters': {'type': 'object',
    'properties': {'params': {'type': 'object',
      'properties': {'a': {'type': 'number', 'title': 'A'},
       'b': {'type': 'number', 'title': 'B'}},
      'required': ['a', 'b']}}}}},
 'metadata': {'method': 'get', 'url': 'http://127.0.0.1:8000/add'}}

In [8]:
embedding_function = SentenceTransformerEmbedder(model="all-MiniLM-L6-v2", cache_dir="model")
api_vector_store = ChromaVectorStore(
    name="api_toolbox",
    embedding_function=embedding_function,
    persistent=False
)

INFO:dapr_agents.document.embedder.sentence:Loading SentenceTransformer model from local path: model
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: model
INFO:dapr_agents.document.embedder.sentence:Model loaded successfully.
INFO:dapr_agents.storage.vectorstores.chroma:ChromaVectorStore initialized with collection: api_toolbox


In [9]:
calculator_agent = OpenAPIReActAgent(
    role="Calculator API Assistant",
    instructions=[
        "You are a helpful assistant that can perform calculations by calling REST API endpoints.",
    ],
    name="CalculatorOpenAPIReActAgent",
    spec_parser=spec_parser,
    api_vector_store=api_vector_store,
)

INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.agent.patterns.openapi.react:Setting up VectorToolStore for OpenAPIReActAgent...
INFO:dapr_agents.tool.storage.vectorstore:Adding tools to Vector Tool Store.
INFO:dapr_agents.document.embedder.sentence:Generating embeddings for 5 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:dapr_agents.tool.executor:Tool registered: GetOpenapiDefinition
INFO:dapr_agents.tool.executor:Tool registered: OpenApiCallExecutor
INFO:dapr_agents.tool.executor:Tool Executor initialized with 2 tool(s).
INFO:dapr_agents.agent.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agent.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agent.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal', 'instructions']


In [10]:
calculator_agent.tool_executor.help

                                                  Available Tools                                                  
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                 ┃ Description                                 ┃ Signature                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ GetOpenapiDefinition │                                             │ GetOpenapiDefinition(user_input: str)      │
│                      │ Search the vector store for OpenAPI         │                                            │
│                      │ *operation IDs / paths* most relevant       │                                            │
│                      │ to **one** user task.                       │                                            │
│                      │                                             │                                            │
│                      │ Always call this **once per new task**      │                                            │
│                      │ *before* attempting an                      │                                            │
│                      │ `open_api_call_executor`. Returns up to 5   │                                            │
│                      │ candidate operations.                       │                                            │
│                      │                                             │                                            │
│ OpenApiCallExecutor  │                                             │ OpenApiCallExecutor(path_template: str,    │
│                      │ Execute **one** REST call described by an   │ method: str, path_params: Dict, data:      │
│                      │ OpenAPI operation.                          │ Dict, headers: Optional = None, params:    │
│                      │                                             │ Optional = None, req_kwargs: Any)          │
│                      │ Use this only *after*                       │                                            │
│                      │ `get_openapi_definition` has returned a     │                                            │
│                      │ matching                                    │                                            │
│                      │ `path_template`/`method`.                   │                                            │
│                      │                                             │                                            │
│                      │ Authentication: merge `auth_header` given   │                                            │
│                      │ at agent-init time with                     │                                            │
│                      │ any per-call `headers` argument (per-call   │                                            │
│                      │ overrides duplicates).                      │                                            │
│                      │                                             │                                            │
└──────────────────────┴─────────────────────────────────────────────┴────────────────────────────────────────────┘

In [11]:
prompt = "Precisely calculate with available tools: 2 + 2, 5*5, 2.4 / 2, -34 + 10, and 2.9283 * 2.23234"

In [12]:
similar = calculator_agent.tool_vector_store.get_similar_tools(prompt)

INFO:dapr_agents.tool.storage.vectorstore:Searching for tools similar to query: Precisely calculate with available tools: 2 + 2, 5*5, 2.4 / 2, -34 + 10, and 2.9283 * 2.23234
INFO:dapr_agents.document.embedder.sentence:Generating embeddings for 1 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
similar

{'ids': [['13944680-4c8e-4ff6-9352-c35a54a795f8',
   'b941e848-c467-4642-b0b7-d67a80ae37de',
   'ca0da60b-0710-4667-a95a-54b4f5817e47',
   'e1d8b739-0ea2-4c39-9473-74a32a2a8b3c']],
 'embeddings': None,
 'documents': [["multiply: Multiply two numbers and return the result.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'required': ['a', 'b']}}}",
   "subtract: Subtract b from a and return the result.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'required': ['a', 'b']}}}",
   "divide: Divide a by b and return the result. Raises an error if b is 0.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'required': ['a', 'b']}}}",

In [14]:
similar["documents"][0]

["multiply: Multiply two numbers and return the result.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'required': ['a', 'b']}}}",
 "subtract: Subtract b from a and return the result.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'required': ['a', 'b']}}}",
 "divide: Divide a by b and return the result. Raises an error if b is 0.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'required': ['a', 'b']}}}",
 "add: Add two numbers and return the result.. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 're

In [15]:
similar["metadatas"][0]

[{'url': 'http://127.0.0.1:8000/multiply',
  'name': 'multiply',
  'method': 'get'},
 {'url': 'http://127.0.0.1:8000/subtract',
  'name': 'subtract',
  'method': 'get'},
 {'url': 'http://127.0.0.1:8000/divide', 'method': 'get', 'name': 'divide'},
 {'method': 'get', 'name': 'add', 'url': 'http://127.0.0.1:8000/add'}]

In [16]:
await calculator_agent.run(prompt)

INFO:dapr_agents.agent.patterns.react.base:Iteration 1/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
Precisely calculate with available tools: 2 + 2, 5*5, 2.4 / 2, -34 + 10, and 2.9283 * 2.23234

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Executing GetOpenapiDefinition with arguments {'user_input': 'calculate mathematical expression'}
INFO:dapr_agents.tool.executor:Running tool (auto): GetOpenapiDefinition
INFO:dapr_agents.tool.storage.vectorstore:Searching for tools similar to query: ['calculate mathematical expression']
INFO:dapr_agents.document.embedder.sentence:Generating embeddings for 1 input(s).


Thought: To perform precise calculations, I'll call the calculation API for each operation. First, I will use the `GetOpenapiDefinition` tool to obtain the relevant API operation details for performing calculations.
Action: {"name": "GetOpenapiDefinition", "arguments": {"user_input": "calculate mathematical expression"}}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:No action specified; continuing with further reasoning.
INFO:dapr_agents.agent.patterns.react.base:Iteration 8/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


Thought: The precise results for the calculations are as follows:
- \(2 + 2 = 4.0\)
- \(5 \times 5 = 25.0\)
- \(2.4 \div 2 = 1.2\)
- \(-34 + 10 = -24.0\)
- \(2.9283 \times 2.23234 = 6.536961222000001\)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Agent provided a direct final answer.


Thought: Answer: The precise results for the calculations are:
- \(2 + 2 = 4.0\)
- \(5 \times 5 = 25.0\)
- \(2.4 \div 2 = 1.2\)
- \(-34 + 10 = -24.0\)
- \(2.9283 \times 2.23234 = 6.536961222000001\)

--------------------------------------------------------------------------------

assistant:
The precise results for the calculations are:
- \(2 + 2 = 4.0\)
- \(5 \times 5 = 25.0\)
- \(2.4 \div 2 = 1.2\)
- \(-34 + 10 = -24.0\)
- \(2.9283 \times 2.23234 = 6.536961222000001\)


'The precise results for the calculations are:\n- \\(2 + 2 = 4.0\\)\n- \\(5 \\times 5 = 25.0\\)\n- \\(2.4 \\div 2 = 1.2\\)\n- \\(-34 + 10 = -24.0\\)\n- \\(2.9283 \\times 2.23234 = 6.536961222000001\\)'